# Mi proyecto en Python

Es un diccionario personalizado. Quiero subirlo a [mi portafolio en github](https://github.com/losataschu).

Primero defino la función que me ayuadará a determinar si acaso una entrada tiene una, dos o más palabras.
Testeé esa función con la lista `mis palabras`:

In [3]:
mis_palabras = ['abarbeiten', 'die Wertschöpfung', 'etwas in Worte fassen']

def contarPalabras(texto):
  if len(texto) == 0:
    return 0
  palabras = 1
  for caracter in texto:
    if caracter == ' ' and texto[0] != caracter and texto[-1] != caracter:
      palabras += 1
  return palabras


Luego voy a adjuntar los archivos que tengo el drive:

In [5]:
# solo para drive
# from google.colab import drive
# drive.mount('/content/drive')

Las librerías que uso para crear mi diccionario: `json` y `pandas`.

In [5]:
import json
import pandas as pd

## Resumen de mis clases

1. Entrada
    1. Texto
    5. Fecha
    4. Categoría
3. Sustantivo
    * Artículo
50. Diccionario
    + Se crea a partir de las entradas
    + Puede agregar entradas
    + Debe permitir mostrar sublistas filtrando por diferentes criterios

Tengo una clase global para una entrada en mi diccionario, `Entrada`, que consiste de un texto, una fecha y una categoría.
La subclase `Sustantivo` además tiene el artículo que lo diferencia del resto de las entradas, que van a ser palabras solas o frases compuestas:

In [7]:
#Codigo Entrada
class Entrada:
  def __init__(self, texto, fecha, categoria):
    self.texto = texto
    self.fecha = fecha
    self.categoria = categoria
  def __str__(self):
    return self.texto
  def __repr__(self):
    return f"{self.texto}"

#Codigo Sustantivo
class Sustantivo(Entrada):
  def __init__(self, articulo, texto, fecha, categoria):
    super().__init__(texto, fecha, categoria)
    self.articulo = articulo
  def __repr__(self):
    return f"{self.articulo} {self.texto}"



In [192]:
# Codigo Clase Diccionario
class Diccionario:
  def __init__(self, direccion):
    self.direccion = direccion
    self.listado = self.crear_listado()

  def crear_listado(self):
    with open(self.direccion, 'r') as file:
      mis_entradas = json.load(file)
      #print(mis_entradas)
      entradas_con_categoria = []
      for entrada in mis_entradas["Entradas"]:
        #print(entrada["palabra"])
        if contarPalabras(entrada["palabra"]) == 2:
          entradas_con_categoria.append(Sustantivo(articulo=entrada["palabra"][0:3], texto=entrada["palabra"][4:], fecha=entrada["fecha"], categoria="Sustantivo"))
        else:
          entradas_con_categoria.append(Entrada(texto=entrada["palabra"], fecha=entrada["fecha"], categoria="Otra"))
      return entradas_con_categoria


In [194]:
class Tabla:
  def __init__(self, listado):
    self.listado = listado
    self.tabla = self.crear_tabla()
    
# Crear una tabla ocupando las entradas del diccionario
  def crear_tabla(self):
    enumeracion = []
    coleccion = []
    i = 1
    entrada_i = "Entrada "
    for entrada in self.listado:
      enumeracion.append(entrada_i + str(i))
      # TODO: agregar una excepcion para regular errores
      fecha_legible = f'{entrada.fecha[0]}-{entrada.fecha[1]}-{entrada.fecha[2]}'
      linea = [entrada.__repr__(), fecha_legible, entrada.categoria]
      coleccion.append(linea)
      i += 1
    return pd.DataFrame(coleccion, index=enumeracion, columns=["Palabra", "Fecha", "Categoria"])


In [221]:
def isSustantivo(entrada):
    if contarPalabras(entrada.texto) == 2:
      Sustantivo(entrada.texto[0], entrada.texto[1], entrada.fecha)

# esta es la función que filtrará las tablas y entrega los resultados
def filtrarTabla(tabla, criterio, valor):
    tabla_filtrada = tabla.copy()
    if criterio == "categoria":
      return tabla.loc[lambda df: df["Categoria"] == valor, :]
    elif criterio == "letra_inicial":
      for i in tabla.loc[:,"Palabra"]:
        if contarPalabras(i) == 2 and i[4] != valor.capitalize():
          idx = tabla.index[tabla["Palabra"]==i].tolist()
          tabla_filtrada = tabla_filtrada.drop(idx)
        elif contarPalabras(i) == 1 and i[0] != valor:
          idx = tabla.index[tabla["Palabra"]==i].tolist()
          tabla_filtrada = tabla_filtrada.drop(idx)
      return tabla_filtrada
    else:
      return "Para tal criterio no hay nada aun"


In [223]:
nuevo_diccionario = Diccionario('entradas_reducidas.json')
mis_entradas = nuevo_diccionario.listado
#print(mis_entradas)
#print(nuevo_diccionario.listado[0].fecha)
#print(nuevo_diccionario.listado[0].__str__())
#print(nuevo_diccionario.listado[1].__repr__())
#print(nuevo_diccionario.listado[1].articulo)

In [225]:
nueva_tabla = Tabla(mis_entradas)
#print(nueva_tabla.tabla)
tabla_entradas = nueva_tabla.tabla
encabezado = tabla_entradas.head(10)
print(encabezado)

                    Palabra       Fecha   Categoria
Entrada 1        abarbeiten  29-11-2022        Otra
Entrada 2          abfangen   24-5-2023        Otra
Entrada 3       die Abfolge  12-12-2022  Sustantivo
Entrada 4          abgelten  28-12-2022        Otra
Entrada 5         abgrenzen   24-5-2023        Otra
Entrada 6   die Besteuerung   7-12-2022  Sustantivo
Entrada 7      bestmoeglich  13-12-2022        Otra
Entrada 8          bewirten  29-12-2022        Otra
Entrada 9      durchreichen  13-12-2022        Otra
Entrada 10      einheitlich  21-12-2022        Otra


In [227]:
filtrarTabla(encabezado, "letra_inicial", "a")

,Palabra,Fecha,Categoria
Entrada 1,abarbeiten,29-11-2022,Otra
Entrada 2,abfangen,24-5-2023,Otra
Entrada 3,die Abfolge,12-12-2022,Sustantivo
Entrada 4,abgelten,28-12-2022,Otra
Entrada 5,abgrenzen,24-5-2023,Otra


## Próximos pasos:

- Filtrar según criterios:
  1. Alfabético
  2. Por fecha
  3. ~~Por categoría~~
- A~nadir palabras al archivo JSON.
- ~~¿Crear una nueva clase *Tabla*?~~

## Ideas adicionales:
+ Crear el archivo JSON con pandas en mi clase `Diccionario` y crear una tabla que se pueda visualizar mejor.